In [1]:
#El Mehdi CHOUHAM        version 1
# mehdichouham@gmail.com for Tictactrip 

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.graph_objects as go

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ARDRegression

In [3]:
ticket_data=pd.read_csv('./data/ticket_data.csv', sep=',', index_col='id')
stations=pd.read_csv('./data/stations.csv', sep=',', index_col='id')
providers=pd.read_csv('./data/providers.csv', sep=',', index_col='id')
cities=pd.read_csv('./data/cities.csv', sep=',', index_col='id')

## Summary :

1 - Dataset review
 <br>2 - Data extraction exercise
 <br>&nbsp;&nbsp;2 - 1 - Prices
 <br>&nbsp;&nbsp;2 - 2 - Durations
 <br>&nbsp;&nbsp;2 - 3 - Price and duration difference per transport mean and per travel range
 <br>3 - Bonus
 <br>&nbsp;&nbsp;3 - 1 - Kilometer price per company
 <br>&nbsp;&nbsp;3 - 1 - New trainline !

## 1 - Datasets review

In [4]:
providers.head()

,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
id,,,,,,,,,
9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
11,3,NaN,infobus,Infobus,False,False,False,False,bus
12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [5]:
ticket_data.head(2)

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
id,,,,,,,,,,,
6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542


In [6]:
stations.head(2)

,unique_name,latitude,longitude
id,,,
1,Aalen (Stuttgarter Straße),48.835296,10.092956
2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883


In [7]:
providers.head(2)

,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
id,,,,,,,,,
9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus


In [8]:
cities.head(2)

,local_name,unique_name,latitude,longitude,population
id,,,,,
5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0


## 2 - Data extraction exercise

### 2 - 1 - Prices

In [9]:
df = ticket_data.merge(providers['fullname'], left_on='company', right_on='id', how='left')
df = df.merge(stations['unique_name'], left_on='o_city', right_on='id', how='left').merge(stations['unique_name'], left_on='d_city', right_on='id', how='left').rename(index=str,
                columns={"unique_name_x": "o_station_name", "unique_name_y" : "d_station_name", "price_in_cents" : "price_in_euros"})

df = df.drop(columns=['arrival_ts','company', 'o_station', 'd_station', 'o_city', 'd_city', 'departure_ts', 'search_ts', 'middle_stations', 'other_companies'])

df.price_in_euros = round(df.price_in_euros/100, 5)

df.describe().loc[['mean','min','max']]

,price_in_euros
mean,43.827111
min,3.000000
max,385.500000


### 2 - 2 - Durations

In [10]:
ticket_data['duration']=pd.to_datetime(ticket_data['arrival_ts'])-pd.to_datetime(ticket_data['departure_ts'])

In [11]:
ticket_data.loc[:,['price_in_cents','duration']].describe().loc[['mean','min','max']]

,price_in_cents,duration
mean,4382.711061,0 days 07:04:37.247600043
min,300.000000,0 days 00:20:00
max,38550.000000,20 days 12:51:00


### 2 - 3 - Price and duration difference per transport mean and per travel range

In [12]:
 def absc_curvi(o_lat, d_lat, delta_long) :
    return m.acos( m.sin(o_lat)*m.sin(d_lat) + m.cos(o_lat)*m.cos(d_lat)*m.cos(delta_long))

In [13]:
r_terre = 6378137
to_rad = np.pi/180

#Calcul et ajout de la distance à partir de l'abscisse curviligne 

coords = ticket_data.merge(stations, how='left', right_on='id', left_on='o_station').rename(index=str, columns={"longitude": "o_longitude", "latitude": "o_latitude", "unique_name": "o_station"})
coords = coords.merge(stations, how='left', right_on='id', left_on='d_station').rename(columns={"longitude": "d_longitude", "latitude": "d_latitude", "unique_name": "d_station"})

coords['delta_long'] = coords.d_longitude - coords.o_longitude
coords['absc_curvi'] =  np.arccos( np.sin(coords.o_latitude*to_rad)*np.sin(coords.d_latitude*to_rad) + np.cos(coords.o_latitude*to_rad)*np.cos(coords.d_latitude*to_rad)*np.cos(coords.delta_long*to_rad))
coords['distance'] = r_terre * coords['absc_curvi'] /1000

coords = coords.drop(columns=[ 'middle_stations', 'o_station', 'd_station', 'departure_ts', 'arrival_ts', 'search_ts', 'delta_long', 'o_latitude', 'o_longitude','d_latitude', 'd_longitude', 'other_companies'])

# coords[coords.o_name == "Massy-Palaiseau"][coords.d_name == "Gare Lille-Europe"] matches google maps distance !!

In [14]:
# Prise en compte du type de transport
trajets = coords.merge(providers['transport_type'], left_on='company', right_on='id', how='left').drop(columns=[ 'company', 'o_city', 'd_city'])
trajets

,price_in_cents,duration,absc_curvi,distance,transport_type
0,4550,0 days 06:10:00,NaN,NaN,carpooling
1,1450,0 days 17:50:00,0.079134,504.730115,bus
2,7400,1 days 07:57:00,0.079158,504.883406,train
3,13500,0 days 21:35:00,0.079158,504.883406,train
4,7710,0 days 21:46:00,0.079158,504.883406,train
...,...,...,...,...,...
74163,2590,0 days 13:30:00,0.053967,344.207811,bus
74164,2700,0 days 06:30:00,0.053321,340.090219,bus
74165,3000,0 days 06:45:00,0.053878,343.643896,bus
74166,2880,0 days 07:45:00,0.053321,340.090219,bus


In [15]:
trajets_0_200 = trajets[trajets['distance'] <= 200]
trajets_201_800 = trajets[(trajets['distance'] > 200) & (trajets['distance'] < 800)]
trajets_800_2000 = trajets[(800 < trajets['distance']) & (trajets['distance'] < 2000)]
trajets_over_2000 = trajets[trajets['distance'] > 2000]

In [16]:
print("Le plus grand trajet est : ", trajets.distance.max(), "Km !\n")

Le plus grand trajet est :  1867.5584602995452 Km !



### Paths per travel ranges :

In [17]:
transport_types = []; list_0_200 = []; list_201_800 = []; list_800_2000 = []; list_over_2000 = [];

for each in pd.unique(trajets['transport_type']) :
    transport_types.append(each)

for each in transport_types:
    seconds =  trajets_0_200.duration[trajets_0_200['transport_type'] == each].mean()
    list_0_200.append([trajets_0_200.price_in_cents[trajets_0_200['transport_type'] == each].mean(), seconds.total_seconds()]) 

for each in transport_types:
    seconds =  trajets_201_800.duration[trajets_201_800['transport_type'] == each].mean()
    list_201_800.append([trajets_201_800.price_in_cents[trajets_201_800['transport_type'] == each].mean(), seconds.total_seconds()])    

for each in transport_types:
    seconds =  trajets_800_2000.duration[trajets_800_2000['transport_type'] == each].mean()
    list_800_2000.append([trajets_800_2000.price_in_cents[trajets_800_2000['transport_type'] == each].mean(), seconds.total_seconds()])    
    
for each in transport_types:
    seconds =  trajets_over_2000.duration[trajets_over_2000['transport_type'] == each].mean()
    list_over_2000.append([trajets_over_2000.price_in_cents[trajets_over_2000['transport_type'] == each].mean(), seconds.total_seconds()])    

averages = np.array([list_0_200, list_201_800, list_800_2000, list_over_2000])    

In [ ]:
travel_ranges = ['from 0 Km to 200 Km', '201 Km to 800 Km', '801 Km to 2000 Km'] #, 'over 2000 Km'], dismissed
fig_1 = go.Figure()
fig_1.data = []; fig_1.layout={}  #reset


for idx, each_transport in enumerate(transport_types) :
    fig_1.add_trace(go.Scatter(x = travel_ranges, y = averages[:, idx, 0]/100, name=each_transport))

fig_1.update_layout(title='Average Price per travel range per transport mean',
                   xaxis_title='Travel ranges',
                   yaxis_title='Price (in euros)')
fig_1.show()

In [ ]:
fig_2 = go.Figure()
fig_2.data = []; fig_2.layout = {} #reset

for idx, each_transport in enumerate(transport_types) :
    fig_2.add_trace(go.Scatter(x = travel_ranges, y = averages[:, idx, 1]/60, name=each_transport))

fig_2.update_layout(title='Average Duration per travel range per transport mean',
                   xaxis_title='Travel ranges',
                   yaxis_title='Duration (in minutes)')
fig_2.show()

## 3 - Bonus

### 3 - 1 - Kilometer price per company

In [19]:
company_info = 'fullname' #or fullname

trajets_companies = coords.merge(providers[['name','fullname']], how='left', right_on='id', left_on='company')
trajets_companies['kilometer_price_in_euros'] =  trajets_companies['price_in_cents']/(100*trajets['distance'])
trajets_companies = trajets_companies.drop(columns=['price_in_cents', 'company', 'duration', 'absc_curvi'])

In [20]:
fig_3 = go.Figure(
    data=[go.Bar(x=trajets_companies.groupby([company_info]).kilometer_price_in_euros.mean().index, y=trajets_companies.groupby([company_info]).kilometer_price_in_euros.mean())],
)

fig_3.update_layout(title="Kilometer price per company",
                   xaxis_title='Providers',
                   yaxis_title='Kilometer price (euros/Km)')
fig_3.show()

### 3 - 2 -  New trainline !

In [21]:
#dataset

cities.loc[(cities.unique_name == 'paris'),'population'] = 2187526 

nl = ticket_data.merge(cities[['unique_name', 'latitude', 'longitude', 'population']], how='left', right_on='id', left_on='o_city').rename(index=str, columns={"longitude": "o_longitude", "latitude": "o_latitude", "unique_name": "o_city_name", 'population': 'o_city_pop'})
nl = nl.merge(cities[['unique_name', 'latitude', 'longitude', 'population']], how='left', right_on='id', left_on='d_city').rename(columns={"longitude": "d_longitude", "latitude": "d_latitude", "unique_name": "d_city_name", 'population': 'd_city_pop'})
nl = nl.merge(providers['fullname'], how='left', right_on='id', left_on='company')

delta_long = nl.d_longitude - nl.o_longitude
nl['distance'] = r_terre * (np.arccos( np.sin(nl.o_latitude*to_rad)*np.sin(nl.d_latitude*to_rad) + np.cos(nl.o_latitude*to_rad)*np.cos(nl.d_latitude*to_rad)*np.cos(delta_long*to_rad)))/1000

nl = nl.rename(columns={'price_in_cents' : 'price_in_euros'})
nl['price_in_euros'] = nl['price_in_euros']/100

nl['month'] = pd.DatetimeIndex(nl['departure_ts']).month
nl['hour'] = pd.DatetimeIndex(nl['departure_ts']).hour
nl['delta_purchase_hours'] = (pd.to_datetime(nl['departure_ts'])-pd.to_datetime(nl['search_ts'])).dt.total_seconds()/3600
nl['duration'] = nl['duration'].dt.total_seconds()/60

nl = nl.drop(columns=[ 'middle_stations', 'o_station', 'd_station', 'departure_ts', 'arrival_ts', 'search_ts', 'o_latitude', 'o_longitude','d_latitude', 'd_longitude', 'other_companies', 'o_city', 'd_city', 'company'])

In [22]:
#we only keep, datas we think are relevant for our prediction; here :
        # origine city and destination city names and populations, the month and the hour for the travel, company and the search ts
nl = nl.dropna()
nl=nl.reindex(columns=list(nl.columns)[1:]+[list(nl.columns)[0]])
nl.head(2)

,duration,o_city_name,o_city_pop,d_city_name,d_city_pop,fullname,distance,month,hour,delta_purchase_hours,price_in_euros
0,370.0,orleans,114977.0,montpellier,275318.0,Blablacar,503.760865,10,14,301.774631,45.5
1,1070.0,orleans,114977.0,montpellier,275318.0,Ouibus,503.760865,10,13,300.856730,14.5


### Encoding and training

In [23]:
encod = OneHotEncoder(handle_unknown='ignore')
nl_str = nl[['o_city_name','d_city_name', 'fullname']]

encoded_nl_str = encod.fit_transform(nl_str)

encoded_nl = pd.concat((pd.DataFrame(encoded_nl_str.toarray(), index=nl.index), nl[[ 'duration', 'o_city_pop', 'd_city_pop', 'distance', 'month', 'hour','delta_purchase_hours','price_in_euros']]), axis=1) 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(encoded_nl.iloc[:,:-1],encoded_nl.iloc[:,-1], test_size=0.2, random_state=42, shuffle=True)

In [25]:
%%time

clf = ARDRegression()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

Wall time: 7.33 s


0.7879031978122739

### Prediction

An interesting trip we can predict the price of is a high speed trainline between Lyon and Bordeaux, two big cities to which people usually travel to by plane for the lack of a 'TGV'

We provide :
- the duration
- both cities populations and names
- the month of departure
- the hour of departure
- the time difference between the ticket purchase and the departure in minutes.

In [26]:
def f(V):
    return np.concatenate((encod.transform(np.array(V[:3]).reshape((1,-1))).toarray()[0], V[3:])).reshape((1,-1))
def predict(trip):
    return round(float(clf.predict(f(trip))), 2)

In [27]:
%%time

trip = ['lyon',      #o_city_name
                 'bordeaux',  #d_city_name
                 'TGV',       #provider fullname
                 513275,      #duration
                 249712,      #o_city_pop
                 1122005,     #d_city_name
                 800,         #distance
                 2,           #month
                 12,           #hour
                 80]          #delta_purchase_hours
    

print('Le prix d\'un voyage de ', trip[1], ' vers ', trip[2], 'est de : \n', predict(trip), 'euros ! \n')

Le prix d'un voyage de  bordeaux  vers  TGV est de : 
 110.64 euros ! 

Wall time: 3.01 ms


### Other interesting ideas :

- Group by paths, for a interesting paths, plot price curves (+ / time lapses)
- Group by paths, mean price for transportation_mean/paths, ( mean price for class_paths)
- Joint company & provider for same class, (donnée inconnues ?)
- Draw coordinates on map
- Correlation between number of travels per region
- Pour une nouvelle ligne, on peut estimer le prix en fonction du prix du kilometre, denivelée des coordonnées ?; populations des villes d'orignies/ destination. + période creuse 
- Utilisation de labelles d'édition d'étiquette/A* pour trouver le trajet le plus court d'une station 1 à une station 2 à partir de la liste des stations (théorie des graphes)

### Thank you !